# Two-Coloured Partitions
Compare Theorem 3.2.1 in Matousek: Using the Borsuk-Ulam Theorem

This example uses bipartite perfect matching to connect pairs of points such that the connections do not intersect. (See below for details.)

In [ ]:
import numpy as np
import networkx as nx
from math import sqrt

# Gurobi constants
from gurobipy import GRB

# Use Matplotlib for plotting; enable svg backend
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from matplotlib import pyplot as plt

# GraphILP API: import networkx graphs and use bipartite perfect matching
from graphilp.imports import networkx as imp_nx
from graphilp.matching import perfect_bipartite

In [ ]:
# Create 2N points in the plane (N of each of two colours)
N = 35

In [ ]:
points = np.random.rand(4,N)

Here are 2N random points in the plane, half of them in one colour, the other half in another.
Try connecting each point of one colour to one point of the other colour by a line such that the resulting lines do not intersect.

In [ ]:
plt.scatter(points[0], points[1])
plt.scatter(points[2], points[3]);

Let's solve this problem by perfect bipartite matching.
Set up a graph whose vertices are given by the points, the colours will be used as a bipartition.
There will be edges between each pair of points of different colour weighted by their Euclidean distance.

In [ ]:
G = nx.empty_graph(2*N)

Create the edges:

In [ ]:
edges = []

for p in range(N):
    for q in range(N):
        edges.append((p, N+q, {'weight':sqrt((points[0][p] - points[2][q]) ** 2 + (points[1][p] - points[3][q]) ** 2)}))

In [ ]:
G.add_edges_from(edges)

Set up the bipartite perfect matching problem using GraphILP API:

In [ ]:
optG = imp_nx.read(G)

In [ ]:
model = perfect_bipartite.create_model(optG, range(N), direction=GRB.MINIMIZE)

Find a perfect matching of minimal weight:

In [ ]:
model.optimize()

In [ ]:
matching = perfect_bipartite.extract_solution(optG, model)

Plot the result.
Note how the lines are connecting points of different colour, yet are not intersecting.

In [ ]:
plt.scatter(points[0], points[1])
plt.scatter(points[2], points[3])
for match in matching:
    plt.plot([points[0][match[0]], points[2][match[1]-N]], [points[1][match[0]], points[3][match[1]-N]], 'k');